<a href="https://colab.research.google.com/github/iyaja/GPT-2-PyTorch/blob/master/GPT_2_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Build the AI that's "Too Dangerous to Release"



Welcome to the companion notebook tutorial for the [OpenAI GPT-2 artcle on floydhub](https://blog.floydhub.com/[ARTICLE NAME]).  Today, you'll create your very own copy of the state of the art language model that drove the internet mad: OpenAI's generative pretrained transformer.

The code we will use is heavily based on huggingface's [`pytorch-pretrained-bert` GitHub repo](https://github.com/huggingface/pytorch-pretrained-BERT). Be sure to check it out, they have top-quality implementations of all the latest and greatest NLP models, as well as fantastic documentation.

One more thing. If you're not running this notebook on FloydHub, [RUN IT ON FLOYDHUB!](https://)

The first step is to download and packages that will help us run the code (this includes the pretrained gpt-2 model).

In [0]:
! pip install pytorch-pretrained-bert
! pip install spacy ftfy==4.4.3
! python -m spacy download en

Next, we'll import the required packages into python. Here's we're getting a little specific on what sub-modules we're using, but feel free to run something like `from pytorch_pretrained_bert import *` if you're just experimenting.

In [0]:
import torch
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

import argparse
import logging
from tqdm import trange

import torch
import torch.nn.functional as F
import numpy as np

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


We'll also activate the logger, so that we can see a little more of what's going on apart from the designated outputs. This step is optional.

In [0]:
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

All NLP models need tokens as inputs. Thankfully we don't need to write a tokenizer from scratch, since the  good men and women at huggingface already did that for us! All we have to do is initialize it.

In [0]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

04/20/2019 14:36:04 - INFO - pytorch_pretrained_bert.tokenization_gpt2 -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /root/.pytorch_pretrained_bert/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
04/20/2019 14:36:04 - INFO - pytorch_pretrained_bert.tokenization_gpt2 -   loading merges file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /root/.pytorch_pretrained_bert/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


Of course, that's just the tokenizer. The actual model is whole other animal. Since this tutorial is about GPT-2, we're going to be using a pretrained model (also part of the hugginface package that we installed).

In [0]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

04/20/2019 14:36:04 - INFO - pytorch_pretrained_bert.modeling_gpt2 -   loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin from cache at /root/.pytorch_pretrained_bert/4295d67f022061768f4adc386234dbdb781c814c39662dd1662221c309962c55.778cf36f5c4e5d94c8cd9cefcf2a580c8643570eb327f0d4a1f007fab2acbdf1
04/20/2019 14:36:04 - INFO - pytorch_pretrained_bert.modeling_gpt2 -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /root/.pytorch_pretrained_bert/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.085d5f6a8e7812ea05ff0e6ed0645ab2e75d80387ad55c1ad9806ee70d272f80
04/20/2019 14:36:04 - INFO - pytorch_pretrained_bert.modeling_gpt2 -   Model config {
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "vocab_size": 50257
}



We're **not** training GPT-2. That would take ages. Since we're running infreence on a pretrained model instead, PyTorch requires us to specify that with the `.eval()` function.

Also, we'll be executing the tensor operations on FloydHub's GPUs, so we'll call `.cuda()` as well.

In [0]:
model.eval()
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (1): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (2): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (3): Block(
        (ln_1): BertLayerNorm()
      

This next block is a helper function that help make work predictions. You can ignore reading through it if you feel uncomfortable with the math.

In [0]:
def top_k_logits(logits, k):
    """
    Masks everything but the k top entries as -infinity (1e10).
    Used to mask logits such that e^-infinity -> 0 won't contribute to the
    sum of the denominator.
    """
    if k == 0:
        return logits
    else:
        values = torch.topk(logits, k)[0]
        batch_mins = values[:, -1].view(-1, 1).expand_as(logits)
        return torch.where(logits < batch_mins, torch.ones_like(logits) * -1e10, logits)

Now, this code block is probably the most important in the notebook. We define a sampling function that takes in tokens and spits out predicted tokens.

Essentially, we're calling the forward pass of GPT-2 (stored in `model`), with some additional statistics stuff that helps make sure we can actually predict works.

In [0]:
def sample_sequence(model, length, start_token=None, batch_size=None, context=None, temperature=1, top_k=0, device='cuda', sample=True):
    if start_token is None:
        assert context is not None, 'Specify exactly one of start_token and context!'
        context = torch.tensor(context, device=device, dtype=torch.long).unsqueeze(0).repeat(batch_size, 1)
    else:
        assert context is None, 'Specify exactly one of start_token and context!'
        context = torch.full((batch_size, 1), start_token, device=device, dtype=torch.long)
    prev = context
    output = context
    past = None
    with torch.no_grad():
        for i in trange(length):
            logits, past = model(prev, past=past)
            logits = logits[:, -1, :] / temperature
            logits = top_k_logits(logits, k=top_k)
            log_probs = F.softmax(logits, dim=-1)
            if sample:
                prev = torch.multinomial(log_probs, num_samples=1)
            else:
                _, prev = torch.topk(log_probs, k=1, dim=-1)
            output = torch.cat((output, prev), dim=1)
    return output

And now that we're done with the heavy lifting, the fun begins!

Here are a few numbers that you might like tweaking. They're all pretty self-explanatory.

In [0]:
sequence_length = 100
number_of_samples = 3

We're done! Run this code block to generate some funky AI text.

Note: you'll have to prompt the model with text, and it will continue writing.

Also note: This model won't perform nearly as well as the one on [the OpenAI blogpos](https://openai.com/blog/better-language-models/)t. In case you didn't know, [they didn't release that model](https://thegradient.pub/openai-please-open-source-your-language-model/).

In [0]:
context_tokens = []
raw_text = input("Model prompt >>> ")

while not raw_text:
  print('Prompt should not be empty!')
  raw_text = input("Model prompt >>> ")

context_tokens = tokenizer.encode(raw_text)
generated = 0

for _ in range(number_of_samples):
  out = sample_sequence(
      model=model, length=sequence_length,
      context=context_tokens, batch_size=512
  )
  out = out[:, len(context_tokens):].tolist()
  for i in range(512):
    generated += 1
    text = tokenizer.decode(out[i])
    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
    print(text)
  print("\n\n"+ "=" * 80)

Model prompt >>> Who is Kyle Rayner? Kyle Rayner is


  0%|          | 0/100 [00:00<?, ?it/s]

======================================== SAMPLE 1 ========================================
 a forty-six year-old Evangelical man living in Arizona. His wife Adriane has three children and spent the past three years (and a half) involved in marriage ceremonies and marriages. As a young man mom, Kyle makes his living blogging about how the Church teaches you, evangelistically. In the article, Kyle complains about being upset with his wife all day long, with giving away all her college scholarships, eating by herself, having sex outside – all while being dominated by a male authority
======================================== SAMPLE 2 ========================================
 a victim of a strange outbreak of part-time illness that has cast the Ashoka Muslim into a permanent discomfiting exile and pursued a different sort of regular life. Both Jason and Sho are all is once again lost to a single narrative in which Kyle is seemingly in immediate danger above all else. The guns from the acade

  0%|          | 0/100 [00:00<?, ?it/s]

======================================== SAMPLE 513 ========================================
 a Mayo Clinic product that bragged about being a kettle snake in 2007. He is a guesswork, but he says he is:

[14]

[15]

[16]

[17]

[18]

[19] In a competition where Eddie Alvarez defeated Tim Kennedy, making that record sixth to the ever Batman-ranked Faber we'd hate to see him go in a loss … not sure how he gets there, maybe his line
======================================== SAMPLE 514 ========================================
 the domain owner, owner, security member and sone. Most people assume that there are 2 inches, 1 inch, and 1 foot of hair in Kyle Rayner's 10 foot wide hand. But, if you look closely at the picture listed below, actions like flying, assembling, teaching John Edmonds and other managers, being an SIS member, being a summer camps volunteer, abusing rashers, and fondling, are not related to Kyle Rayner's behavior at his DC Camp and
========================================

100%|██████████| 100/100 [00:11<00:00,  6.15it/s]


======================================== SAMPLE 1025 ========================================
 a well-respected NFL quarterback in prep football, a strong offensive line, and a troubled future in the NFL. In the game of football we've seen him help fellow youngsters all over the nation get the help they need. You don't have to be a physical prospect to get his name on the team. In a four-year game, no one is going to look away." -- Eric Pinkston, Ret. Day Contributor


This Puck Daddy Demo Nobody read that too hard
======================================== SAMPLE 1026 ========================================
 Cody's buddy in recent Days of Heaven games. He helped create such a devastating Demon Zone artifact in Dead Zone (US have never helped out with the artifact project)

big N 52nd on earth game: rocks a volleyball, level, and various other cool stuff, it you won, baby

NBA 2K19 game: QUAKED ut 5M MM2K18s with "GS drag striker" retarded works

PHOTOS.: Bottom massage scene, proven ar